In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
import numpy as np
import random
import matplotlib.pyplot as plt

from utils.loader import DataLoader
from models.effnet_encoder import EffnetEncoder
from models.mtl_framework import MTLFramework
from utils import tools, config

In [2]:
# Set configs
batch_size = 32
batch_size_val = 32
num_train, num_val, num_test = config.config['num_train'], config.config['num_val'], config.config['num_test']
img_height, img_width, channels = config.config['input_shape']

In [3]:
# Load our data pipeline
loader = DataLoader(batch_size=batch_size, batch_size_val=batch_size_val)

# Train set
img_ds = loader.get_image_ds().repeat()
masks_ds = loader.get_mask_ds().repeat()
label_ds = loader.get_binary_ds().repeat()
bbox_ds = loader.get_bboxes_ds().repeat()

# Validation set
img_ds_val = loader.get_image_ds(val=True).repeat()
masks_ds_val = loader.get_mask_ds(val=True).repeat()
label_ds_val = loader.get_binary_ds(val=True).repeat()
bbox_ds_val = loader.get_bboxes_ds(val=True).repeat()

Build our MTL

In [ ]:
### CLEARS OLD MODELS IN CACHE
tf.keras.backend.clear_session()

In [26]:
# Get encoder with attention unit
base_model_name = 'B0'
encoder = EffnetEncoder(base_model_name, (img_height, img_width, channels)).build_encoder_with_attention(trainable=True) # EfficientNet with attention

In [ ]:
encoder.summary()

In [ ]:
# Use our MTL framework to custom build a model
mtl_builder = MTLFramework(encoder, (img_height, img_width, channels)) # We pass the new attention based EfficientNet encoder
mtl_builder.add_segmentation_head()
mtl_builder.add_binary_classification_head(base_model_name, trainable=True)
mtl_builder.add_bbox_classification_head(base_model_name, trainable=True)
model = mtl_builder.build_mtl_model()

In [62]:
model.trainable = True

In [ ]:
model.summary()

Train model

In [29]:
def generator_img():
    ''' Merges together datasets into a unified generator to pass for training '''
    a = img_ds.as_numpy_iterator()
    b = masks_ds.as_numpy_iterator()
    c = label_ds.as_numpy_iterator()
    d = bbox_ds.as_numpy_iterator()
    
    while True:
        X = a.next()
        Y1 = b.next()
        Y2 = c.next()
        Y3 = d.next()
        
        # Regularisation and shuffling
        X, Y1, Y2, Y3 = tools.get_randomised_data([X, Y1, Y2, Y3])
        X, Y1, Y3 = tools.data_augmentation(X, Y1, Y3) # Fix augmentation
        
        yield X, (Y1, Y2, Y3)

In [30]:
def generator_img_val():
    ''' Merges together datasets into a unified generator to pass for training '''
    a = img_ds_val.as_numpy_iterator()
    b = masks_ds_val.as_numpy_iterator()
    c = label_ds_val.as_numpy_iterator()
    d = bbox_ds.as_numpy_iterator()
    
    while True:
        X = a.next()
        Y1 = b.next()
        Y2 = c.next()
        Y3 = d.next()
        
        yield X, (Y1, Y2, Y3)

In [68]:
# Initial training
model.compile(optimizer=keras.optimizers.Adam(),
              loss={'segnet_out' : tf.keras.losses.BinaryCrossentropy(from_logits=True),
                    'bin_class_out' : tf.keras.losses.BinaryCrossentropy(),
                    'bbox_out' : tf.keras.losses.MeanAbsoluteError()},
              loss_weights=[1,1,1/100], # Scale MAE to BC range
              metrics=['accuracy'])

In [ ]:
history = model.fit(generator_img(), validation_data=generator_img_val(), epochs=15, steps_per_epoch=num_train//batch_size, validation_steps=num_val//batch_size_val)

In [ ]:
# Fine-tuning at lower LR
model.compile(optimizer=keras.optimizers.Adam(1e-4),
              loss={'segnet_out' : tf.keras.losses.BinaryCrossentropy(from_logits=True),
                    'bin_class_out' : tf.keras.losses.BinaryCrossentropy(),
                    'bbox_out' : tf.keras.losses.MeanAbsoluteError()},
              loss_weights=[1,1,1/100], # Scale MAE to BC range
              metrics=['accuracy'])

In [ ]:
history = model.fit(generator_img(), validation_data=generator_img_val(), epochs=10, steps_per_epoch=num_train//batch_size, validation_steps=num_val//batch_size_val)

In [ ]:
model.save('model_weights/EffishingNetAtt_Eff')

## Test on test-set

In [5]:
model = tf.keras.models.load_model('model_weights/EffishingNetAtt_Eff')

In [6]:
# Load test-set
img_ds_test = loader.get_image_ds(test_mode=True)
masks_ds_test = loader.get_mask_ds(test_mode=True)
label_ds_test = loader.get_binary_ds(test_mode=True)
bbox_ds_test = loader.get_bboxes_ds(test_mode=True)

In [7]:
# Predict on test-set
seg_pred, bin_pred, bbox_pred = model.predict(img_ds_test, batch_size=10)
seg_pred = tf.where(seg_pred >= 0, 1, 0) # Convert to {0,1} binary classes
bin_pred = np.round(bin_pred) # Round confidence score

bin_acc = np.sum(bin_pred == label_ds_test)/label_ds_test.shape[0]
seg_acc = np.sum(seg_pred == masks_ds_test)/(masks_ds_test.shape[0]*(img_height*img_width))
iou = np.mean(tools.calculate_iou(bbox_ds_test, bbox_pred))
print(f'Binary Acc: {round(bin_acc*100, 3)}%,   Seg Acc: {round(seg_acc*100, 3)}%,    BBox IOU: {round(iou*100, 3)}%')

Binary Acc: 99.322%,   Seg Acc: 95.772%,    BBox IOU: 79.134%


In [ ]:
# Get precision
m = tf.keras.metrics.Precision()
m.update_state(seg_pred, masks_ds_test)
print(f'Precision of network: {m.result().numpy()}')

# Get recall
m = tf.keras.metrics.Recall()
m.update_state(seg_pred, masks_ds_test)
print(f'Recall of network: {m.result().numpy()}')

# Get Dice metric
m = tf.keras.metrics.MeanIoU(num_classes=2)
m.update_state(seg_pred, masks_ds_test)
print(f'Dice score of network: {m.result().numpy()}')

# Get feature maps

In [ ]:
input_img = next(img_ds.as_numpy_iterator()) # Get test image

# Build new model with intermediate layer as output
XX = model.input 
YY = model.layers[-12].output
new_model = tf.keras.Model(XX, YY)

Xresult = new_model.predict(input_img) # Get feature map

# fig, ax = plt.subplots(5, 10, figsize=(15, 7)) # REMOVE?

# for j in range(5):
#     for i in range(10):
#         ax[j][i].imshow(Xresult[0, :, :, j*20 + i + 10])
#         ax[j][i].axis('off')